In [2]:
import numpy as np
import random
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
def create_feature_sets_and_labels(test_size = 0.3):

    # known patterns (5 features) output of [1] of positions [0,4]==1
    features = []
    features.append([[0, 0, 0, 0, 0], [0,1]])
    features.append([[0, 0, 0, 0, 1], [0,1]])
    features.append([[0, 0, 0, 1, 1], [0,1]])
    features.append([[0, 0, 1, 1, 1], [0,1]])
    features.append([[0, 1, 1, 1, 1], [0,1]])
    features.append([[1, 1, 1, 1, 0], [0,1]])
    features.append([[1, 1, 1, 0, 0], [0,1]])
    features.append([[1, 1, 0, 0, 0], [0,1]])
    features.append([[1, 0, 0, 0, 0], [0,1]])
    features.append([[1, 0, 0, 1, 0], [0,1]])
    features.append([[1, 0, 1, 1, 0], [0,1]])
    features.append([[1, 1, 0, 1, 0], [0,1]])
    features.append([[0, 1, 0, 1, 1], [0,1]])
    features.append([[0, 0, 1, 0, 1], [0,1]])
    features.append([[1, 0, 1, 1, 1], [1,0]])
    features.append([[1, 1, 0, 1, 1], [1,0]])
    features.append([[1, 0, 1, 0, 1], [1,0]])
    features.append([[1, 0, 0, 0, 1], [1,0]])
    features.append([[1, 1, 0, 0, 1], [1,0]])
    features.append([[1, 1, 1, 0, 1], [1,0]])
    features.append([[1, 1, 1, 1, 1], [1,0]])
    features.append([[1, 0, 0, 1, 1], [1,0]])

    # shuffle out features and turn into np.array
    random.shuffle(features)
    features = np.array(features)

    # split a portion of the features into tests
    testing_size = int(test_size*len(features))

    # create train and test lists
    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])

    return train_x, train_y, test_x, test_y

In [14]:
train_x, train_y, test_x, test_y = create_feature_sets_and_labels()

x = tf.placeholder('float')
y = tf.placeholder('float')

# classes in our output
n_classes = 2
# iterations and batch-size to build out model
epochs = 1000
batch_size = 4
# Define Artificial Neural Network
def ann_model(data):
    # Input Layer
    input_layer = tf.reshape(data, [-1, 5])
    
    # First Hidden Layer
    hidden_layer_1 = tf.layers.dense(inputs=input_layer,
                                    units=20,
                                    activation=tf.sigmoid)
    # Second Hidden Layer
    hidden_layer_2 = tf.layers.dense(inputs=hidden_layer_1,
                                    units=20,
                                    activation=tf.sigmoid)
    # Output Layer
    output_layer = tf.layers.dense(inputs=hidden_layer_2,
                                  units=2,
                                  activation=None)
    return output_layer

def train_neural_network(x):
    prediction = ann_model(x)

    # formula for cost (error)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )

    # optimize for cost using GradientDescent
    optimizer = tf.train.GradientDescentOptimizer(1).minimize(cost)

    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter('log_ANN_graph', sess.graph)
        # initialize our variables
        sess.run(tf.global_variables_initializer())

        # loop through specified number of iterations
        for epoch in range(epochs):
            epoch_loss = 0
            i=0
            # handle batch sized chunks of training data
            while i < len(train_x):
                start = i
                end = i+batch_size
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])

                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
                epoch_loss += c
                i+=batch_size
                last_cost = c

            # print cost updates along the way
            if (epoch% (epochs/5)) == 0:
                print('Epoch', epoch, 'completed out of',epochs,'cost:', last_cost)
        
        # print accuracy of our model
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))

        #output_weight = sess.run(output_layer['weight'])
        #output_bias = sess.run(output_layer['bias'])
        
        # print predictions using our model
        for i,t in enumerate(test_x):
            print ('prediction for:', test_x[i])
            output = prediction.eval(feed_dict = {x: [test_x[i]]})
            # normalize the prediction values
            print(tf.sigmoid(output[0][0]).eval(), tf.sigmoid(output[0][1]).eval())
    
train_neural_network(x)

('Epoch', 0, 'completed out of', 1000, 'cost:', 1.0824449)
('Epoch', 200, 'completed out of', 1000, 'cost:', 0.0023858943)
('Epoch', 400, 'completed out of', 1000, 'cost:', 0.00084222062)
('Epoch', 600, 'completed out of', 1000, 'cost:', 0.00047860271)
('Epoch', 800, 'completed out of', 1000, 'cost:', 0.00032540574)
('Accuracy:', 1.0)
('prediction for:', [1, 0, 0, 1, 1])
(0.97955632, 0.017272029)
('prediction for:', [1, 0, 0, 1, 0])
(0.011345561, 0.9886331)
('prediction for:', [1, 1, 0, 1, 0])
(0.016894776, 0.98307794)
('prediction for:', [0, 1, 0, 1, 1])
(0.015653733, 0.98374373)
('prediction for:', [0, 0, 0, 0, 0])
(0.0002428725, 0.99978048)
('prediction for:', [1, 1, 0, 0, 0])
(0.02363053, 0.97603691)
